In [2]:
from pymongo import MongoClient
from datetime import datetime
import pandas as pd
pd.set_option('display.max_columns', None)
import json
import os

In [3]:
# Connexion à la base de données MongoDB
client = MongoClient('localhost', 27017)
db = client['foot']
collection = db['matchs']

In [155]:
resultats = collection.find({"Competition": "LaLiga", "Season": "2006-2007"})
#resultats = collection.find({"$expr": {"$gte": [{"$dateFromString": {"dateString": "$Date"}}, datetime(2021, 1, 1)]}})

# Conversion du curseur en dataframe
df = pd.DataFrame(list(resultats))

df.drop('_id', axis=1, inplace=True)
columns_to_flatten = ['Teams', 'Goals', 'xG', 'Manager', 'Captain', 'Possession',
                      'Fouls', 'Corners', 'Crosses', 'Touches', 'Tackles', 'Interceptions',
                      'Aerials Won', 'Clearances', 'Offsides', 'Goal Kicks', 'Throw Ins',
                      'Long Balls', 'Formations']

for col in columns_to_flatten:
    if col in df.columns:
        # Créer une colonne temporaire avec les données extraites de la colonne col
        temp_col = df[col].apply(pd.Series)

        # Renommer les colonnes extraites
        temp_col = temp_col.rename(columns={'Home': col+'_Home', 'Away': col+'_Away'})

        # Insérer les nouvelles colonnes à la position de la colonne col
        df = pd.concat([df.iloc[:, :df.columns.get_loc(col)], temp_col, df.iloc[:, df.columns.get_loc(col)+1:]], axis=1)

# Affichage du dataframe
df.head()

,Teams_Home,Teams_Away,Goals_Home,Goals_Away,Manager_Home,Manager_Away,Venue,Officials,Date,Weekday,Events,Home Team Player Stats,Away Team Player Stats,Home Goal Player Stats,Away Goal Player Stats,Competition,Season,Matchweek,Round
0,Getafe,Athletic Club,0,0,Bernd Schuster,Félix Sarriugarte,Coliseum Alfonso Pérez,{'Referee': 'Ramírez Domínguez'},2006-09-24,Sunday,"[{'Min': 25, 'Team': 'Away', 'Type': 'Substitu...","{'Pato Abbondanzieri': {'Nation': 'ar ARG', '...","{'Aritz Aduriz': {'Nation': 'es ESP', 'Pos': ...","{'Pato Abbondanzieri': {'Nation': 'ar ARG', '...","{'Iñaki Lafuente': {'Nation': 'es ESP', 'Age'...",LaLiga,2006-2007,4,Regular season
1,Celta Vigo,Racing Santander,2,2,Fernando Vázquez,Miguel Ángel Portugal,Estadio de Balaídos,{'Referee': 'Medina Cantalejo'},2007-02-25,Sunday,"[{'Min': 24, 'Team': 'Home', 'Type': 'Penalty ...","{'Jonathan Aspas': {'Nation': 'es ESP', 'Pos'...","{'Gonzalo Colsa': {'Nation': 'es ESP', 'Pos':...","{'José Manuel Pinto': {'Nation': 'es ESP', 'A...","{'Toño': {'Nation': 'es ESP', 'Age': {'Year':...",LaLiga,2006-2007,24,Regular season
2,Villarreal,Osasuna,1,4,Manuel Pellegrini,Cuco Ziganda,El Madrigal,{'Referee': 'Delgado Ferreiro'},2006-12-17,Sunday,"[{'Min': 12, 'Team': 'Away', 'Type': 'Goal', '...","{'Quique Álvarez': {'Nation': 'es ESP', 'Pos'...","{'Enrique Corrales': {'Nation': 'es ESP', 'Po...","{'Mariano Barbosa': {'Nation': 'ar ARG', 'Age...","{'Ricardo': {'Nation': 'es ESP', 'Age': {'Yea...",LaLiga,2006-2007,15,Regular season
3,Real Sociedad,Espanyol,1,1,Miguel Ángel Lotina,Ernesto Valverde,Anoeta,{'Referee': 'Lizondo Cortés'},2006-11-12,Sunday,"[{'Min': 30, 'Team': 'Home', 'Type': 'Goal', '...","{'Ion Ansotegi': {'Nation': 'es ESP', 'Pos': ...","{'Javier Chica': {'Nation': 'es ESP', 'Pos': ...","{'Claudio Bravo': {'Nation': 'cl CHI', 'Age':...","{'Carlos Kameni': {'Nation': 'cm CMR', 'Age':...",LaLiga,2006-2007,10,Regular season
4,Zaragoza,Deportivo La Coruña,1,1,Víctor Fernández,Joaquín Caparrós,La Romareda,{'Referee': 'Álvarez Izquierdo'},2007-01-28,Sunday,"[{'Min': 20, 'Team': 'Away', 'Type': 'Goal', '...","{'Pablo Aimar': {'Nation': 'ar ARG', 'Pos': [...","{'Jorge Andrade': {'Nation': 'pt POR', 'Pos':...","{'César Sánchez': {'Nation': 'es ESP', 'Age':...","{'Dudu Aouate': {'Nation': 'il ISR', 'Age': {...",LaLiga,2006-2007,20,Regular season


In [156]:
df['Teams_Away'].value_counts()

Athletic Club          19
Racing Santander       19
Getafe                 19
Barcelona              19
Sevilla                19
Recreativo             19
Zaragoza               19
Levante                19
Villarreal             19
Atlético Madrid        19
Mallorca               19
Real Sociedad          19
Real Betis             19
Gimnàstic              19
Real Madrid            19
Valencia               19
Deportivo La Coruña    19
Espanyol               19
Osasuna                19
Celta Vigo             19
Name: Teams_Away, dtype: int64

In [157]:
def get_ranking(df, matchweek=None):
    # initialiser le dictionnaire
    ranking = {}

    # itérer sur chaque ligne du dataframe
    for index, row in df.iterrows():
        # récupérer les valeurs des colonnes
        home_team = row['Teams_Home']
        away_team = row['Teams_Away']
        home_score = row['Goals_Home']
        away_score = row['Goals_Away']
        # comparer les scores pour déterminer le gagnant
        if home_score > away_score:
            ranking.setdefault(home_team, 0)
            ranking[home_team] += 3
        elif away_score > home_score:
            ranking.setdefault(away_team, 0)
            ranking[away_team] += 3
        else:
            ranking.setdefault(home_team, 0)
            ranking.setdefault(away_team, 0)
            ranking[home_team] += 1
            ranking[away_team] += 1
    ranking = dict(sorted(ranking.items(), key=lambda item: item[1],reverse=True))

    return ranking